In [ ]:
import pandas as pd
import pymongo
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import text
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# # create a connection to Postgres
# conn = psycopg2.connect(database="postgres", user='postgres', password='Shjais2014', host='127.0.0.1', port='5432')
# conn.autocommit = True
# cursor = conn.cursor()

# # create the database in Postgres
# create_database_sql = open('../sql/create_db.sql','r')
# cursor.execute(create_database_sql.read())

# #If no error print message
# print("Database created successfully........")

# #Closing the connection and database
# conn.close()
# create_database_sql.close()

In [ ]:
csv_file = "../output_data/tweets.csv"
twitter_df = pd.read_csv(csv_file)
twitter_df.head()

In [ ]:
twitter_df.rename(columns = {'Tweets':'tweets'}, inplace = True) 
twitter_df.head()

In [ ]:
# twitter_df = twitter_df.set_index("state")
# twitter_df.head()
twitter_df = twitter_df.drop(columns=['Unnamed: 0'])
twitter_df.head()

In [ ]:
twitter_df = twitter_df.set_index("id")
twitter_df.head()

In [ ]:
# create a connection to Postgres
conn = psycopg2.connect(database="twitter_db", user='postgres', password='Shjais2014', host='127.0.0.1', port='5432')
conn.autocommit = True
cursor = conn.cursor()

In [ ]:
# run the schema for the database using the schema file created and the connection to Postgres
schema_sql = open('../sql/schema.sql','r')
cursor.execute(schema_sql.read())

#If no error print message
print("Database Schema Loaded Successfully (Tables Created)........")

# close the schema file
schema_sql.close()

In [ ]:
#database connection
connection_string ="postgres:Shjais2014@localhost:5432/twitter_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
#confirm tables
engine.table_names()

In [ ]:
### Load DataFrames into database
twitter_df.to_sql(name='twitter_tweets', con=engine, if_exists='append', index = True)

In [ ]:
select_sql = open('../sql/select_df.sql','r')
twit_df = pd.read_sql_query(select_sql.read(), con=engine)

#Close File after use
select_sql.close()

In [ ]:
twit_df

In [ ]:
conn.close()